In [1]:
###################################################################################################
###                             Motif Enrichment                                ###
###################################################################################################

#load libraries
library(tidyverse)
library(devtools)
library(motifmatchr)
library(BiocParallel)
load_all('/home/jpm73279/genome_downloads/BS_genomes/BSgenome.Zm_B73')
load_all("/home/jpm73279/genome_downloads/BS_genomes/BSgenome.Os")
load_all("/home/jpm73279/genome_downloads/BS_genomes/BSgenome.Pm")
load_all("/home/jpm73279/genome_downloads/BS_genomes/BSgenome.Sb")
load_all("/home/jpm73279/genome_downloads/BS_genomes/BSgenome.Uf")

library(Matrix)
library(GenomicAlignments)
library(dplyr)
library(universalmotif)  # manipulating motif representations




── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2          ✔ readr     2.1.4     
✔ forcats   1.0.0.9000     ✔ stringr   1.5.1     
✔ ggplot2   3.4.4          ✔ tibble    3.2.1     
✔ lubridate 1.9.2          ✔ tidyr     1.3.0     
✔ purrr     1.0.2          
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Warning message:
“package ‘devtools’ was built under R version 4.3.1”
Loading required package: usethis



ℹ Loading BSgenome.Zm_B73
Loading required package: BSgenome

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:lubridate’:

    intersect, setdiff, union


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, unio

In [2]:

## Load Markers for All Species
marker_file_paths <- c("/home/jpm73279/genome_downloads/C4_markers/Os.c4_markers.bed",
                "/home/jpm73279/genome_downloads/C4_markers/Pm.c4_markers.bed",
                "/home/jpm73279/genome_downloads/C4_markers/Sb.c4_markers.bed",
                "/home/jpm73279/genome_downloads/C4_markers/Uf.c4_markers.bed",
                "/home/jpm73279/genome_downloads/C4_markers/Zm.c4_markers.bed")

# Function to read and process the file
process_file <- function(file_path) {
  species <- substr(basename(file_path), 1, 2)
  read_delim(file_path, delim = "\t", col_names = c("chrom", "start", "end", "geneID", "name", "type")) %>% 
    dplyr::select("chrom","start","end","geneID","name","type") %>%
    dplyr::mutate(species = (species))
}

# Apply the function to each file and store results in a list
list_of_dataframes <- lapply(marker_file_paths, process_file)

# Optionally combine all dataframes into one if needed
markers <- bind_rows(list_of_dataframes) %>% 
    dplyr::select(geneID,name,type,species)

Rows: 30 Columns: 6
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (4): chrom, geneID, name, type
dbl (2): start, end

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 52 Columns: 6
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (4): chrom, geneID, name, type
dbl (2): start, end

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 33 Columns: 6
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (4): chrom, geneID, name, type
dbl (2): start, end

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 31 Columns: 6

In [3]:
### Load Motifs for Analysis
core_motifs <- read_meme("/scratch/jpm73279/comparative_single_cell/07.call.ACRs/00.data/conserved_consensus_motifs_Fig2B.meme")
de_novo_motifs <- read_meme("/home/jpm73279/genome_downloads/C4_markers/found_motifs/combined_unique_motifs.meme")
ms_de_novo_motifs <- read_meme("/home/jpm73279/genome_downloads/C4_markers/found_motifs/Ms.specific_CGTCGT_motif.comparative.meme")

all_assy_motifs <- c(core_motifs, de_novo_motifs, ms_de_novo_motifs)
all_assy_motifs_converted <- convert_motifs(all_assy_motifs, class = "TFBSTools-PWMatrix")

update_name <- function(pwm) {
    if (!is.null(pwm@name)) {
        pwm@ID <- pwm@name
    }
    return(pwm)
}


### Update the Names to be the actual IDs, as well as convert to PWM matrix List (Nasty as fuck)
library(TFBSTools)
pw_matrix_list_converted <- lapply(all_assy_motifs_converted, update_name)
pw_matrix_list_converted <- lapply(pw_matrix_list_converted, PWMatrixList)
pw_matrix_list_converted <- do.call(c, pw_matrix_list_converted)


#Generate an Index Annotation name for later usage. Will be used because we need to rip these to annotate the Granges objects
#later
extractIDDataFrame <- function(pwMatrixList) {
  ids <- sapply(pwMatrixList@listData, function(item) item@ID)
  ids <- gsub("chr", "", ids, fixed = TRUE)
  indices <- seq_along(ids)
  
  data.frame(group = indices, TF_ID = ids)
}

# Example usage:
# Assuming your PWMatrixList object is named pwMatrixListObject
motif_id_index <- extractIDDataFrame(pw_matrix_list_converted)


Attaching package: ‘TFBSTools’


The following object is masked from ‘package:Matrix’:

    Matrix




In [4]:
sb_acrs <- read_delim("/scratch/jpm73279/comparative_single_cell/07.call.ACRs/copy_over/sb_acr_classification.no_exons.all_ACRs.classified.sorted.bed", col_names = c("chr", "start", "end", "acr_id", "score"))
pm_acrs <- read_delim("/scratch/jpm73279/comparative_single_cell/07.call.ACRs/copy_over/pm_acr_classification.no_exons.all_ACRs.classified.sorted.bed", col_names = c("chr", "start", "end", "acr_id", "score"))
zm_acrs <- read_delim("/scratch/jpm73279/comparative_single_cell/07.call.ACRs/copy_over/zm_acr_classification.no_exons.all_ACRs.classified.sorted.bed", col_names = c("chr", "start", "end", "acr_id", "score"))
uf_acrs <- read_delim("/scratch/jpm73279/comparative_single_cell/07.call.ACRs/copy_over/uf_acr_classification.no_exons.all_ACRs.classified.sorted.bed", col_names = c("chr", "start", "end", "acr_id", "score"))
os_acrs <- read_delim("/scratch/jpm73279/comparative_single_cell/07.call.ACRs/copy_over/os_acr_classification.no_exons.all_ACRs.classified.sorted.bed", col_names = c("chr", "start", "end", "acr_id", "score"))



Rows: 51387 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (3): chr, acr_id, score
dbl (2): start, end

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 65990 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (3): chr, acr_id, score
dbl (2): start, end

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 55504 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (3): chr, acr_id, score
dbl (2): start, end

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 29153 Columns: 5
── Colum

In [5]:
header_bed <- c('chr','start','end','id','score')
zm_acr_intersections <- read_delim("/home/jpm73279/genome_downloads/C4_markers/conservation/zm.all_intersection_acr.intersection.bed", 
            col_names = header_bed) %>% 
    separate(id, into = c("acr", "acr_cell_type_specific_class", "locus"), sep = ";") %>%
    dplyr::mutate(acr_cell_type_specific_class = case_when(acr_cell_type_specific_class == "bundle_sheath,procambial_meristem" ~ "bundle_sheath",
                                                          acr_cell_type_specific_class == "bundle_sheath,procambium" ~ "bundle_sheath", 
                                                          TRUE ~ acr_cell_type_specific_class)) %>% 
    tidyr::separate(locus, into = c("species_other", "loci"), sep = "__",remove = FALSE) %>%
    tidyr::separate(loci, into = c("gene_family", "number"), sep = "_", remove = FALSE)

combined_acrs_marker_info <- left_join(zm_acr_intersections, markers, by = c("locus" = "name")) %>% 
    select(chr:acr_cell_type_specific_class, locus, gene_family, type)


Rows: 146 Columns: 5
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (3): chr, id, score
dbl (2): start, end

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [6]:
set.seed(420)

In [7]:
processPeaks <- function(peaks, genome, motif_IDs) {
    # Convert peaks to GRanges object
    peaks_gr <- GRanges(peaks, seqinfo=seqinfo(genome))
    names(peaks_gr) <- as.character(peaks_gr)
    peaks_gr <- unique(peaks_gr)

    # Get peak sequences
    peak_seqs <- getSeq(genome, peaks_gr)

    # Exclude peaks with any N's
    hasNs <- (alphabetFrequency(peak_seqs)[,"N"] > 0)
    peak_seqs <- peak_seqs[!hasNs]

    # Run matchMotifs
    system.time(
        motif_matches <- matchMotifs(
            pw_matrix_list_converted, 
            peaks_gr, 
            genome = genome, 
            p.cutoff = .0005, # using a stringent p-value
            out="positions"
        )
    )

    # Annotate each GRanges object in the list with the corresponding TF ID
    granges_list <- motif_matches
    for (i in seq_along(granges_list)) {
        granges_list[[i]]$TF_ID <- motif_IDs[i]
        
        overlaps <- findOverlaps(granges_list[[i]], peaks_gr)
        ids <- peaks_gr$acr_id[subjectHits(overlaps)]
        granges_list[[i]]$acr_id <- ids[queryHits(overlaps)]
    }

    return(granges_list)
}


## Run Zea mays

In [8]:
zm_peaks_gr <- GRanges(zm_acrs, seqinfo=seqinfo(BSgenome.Zm_B73))
names(zm_peaks_gr) <- as.character(zm_peaks_gr)
zm_peaks_gr <- unique(zm_peaks_gr)

# get peak sequences, we'll need this later
zm_peak_seqs <- getSeq(BSgenome.Zm_B73, zm_peaks_gr) # <2s


# exclude peaks with any N's, to avoid warnings further down
hasNs    <- (alphabetFrequency(zm_peak_seqs)[,"N"]>0)
addmargins(table(hasNs)) # => only drops 5 sequences

zm_peak_seqs <- zm_peak_seqs[!hasNs]
zm_peak_seqs  <- zm_peak_seqs[!hasNs]


# run: 30k sequences, 100 motifs
system.time(
  motif_matches_zm <- matchMotifs(
    pw_matrix_list_converted, 
    zm_peaks_gr, 
      genome = BSgenome.Zm_B73, 
    p.cutoff = .001, # using a stringent p-value
    out="positions"
))  # <10s on my laptop

motif_IDs <- motif_id_index$TF_ID
zm_granges_list <- motif_matches_zm
# Annotate each GRanges object in the list with the corresponding TF ID
for (i in seq_along(zm_granges_list)) {
    zm_granges_list[[i]]$TF_ID <- motif_IDs[i]
    
    overlaps <- findOverlaps(zm_granges_list[[i]] , zm_peaks_gr)
    ids <- zm_peaks_gr$acr_id[subjectHits(overlaps)]
    zm_granges_list[[i]]$acr_id <- ids[queryHits(overlaps)]
}


hasNs
FALSE   Sum 
55504 55504 

   user  system elapsed 
  5.359   0.149   5.525 

In [200]:
zm_granges_list

GRangesList object of length 28:
[[1]]
GRanges object with 38201 ranges and 3 metadata columns:
          seqnames              ranges strand |     score       TF_ID
             <Rle>           <IRanges>  <Rle> | <numeric> <character>
      [1]     chr1         34510-34515      + |   9.11767         ARF
      [2]     chr1       165528-165533      - |   8.87641         ARF
      [3]     chr1       258456-258461      + |   8.87641         ARF
      [4]     chr1       328049-328054      - |   9.11767         ARF
      [5]     chr1       328333-328338      - |   9.84689         ARF
      ...      ...                 ...    ... .       ...         ...
  [38197]     chr9 162072206-162072211      - |   9.84689         ARF
  [38198]     chr9 162104262-162104267      + |   9.84689         ARF
  [38199]     chr9 162232979-162232984      + |   9.11767         ARF
  [38200]     chr9 162278311-162278316      + |   9.11767         ARF
  [38201]     chr9 162499008-162499013      + |   8.87641       

In [198]:
zm_acrs_TFs <- as_tibble(as.data.frame(zm_granges_list)) %>% 
    tidyr::separate(acr_id, into = c("acr", "acr_cell_type_specific_class"), sep = ";") %>%
    dplyr::mutate(acr_cell_type_specific_class = case_when(acr_cell_type_specific_class == "bundle_sheath,procambial_meristem" ~ "bundle_sheath",
                                                          acr_cell_type_specific_class == "bundle_sheath,procambium" ~ "bundle_sheath", 
                                                          TRUE ~ acr_cell_type_specific_class)) %>% 
    left_join(., combined_acrs_marker_info_subset, by = c("acr", "acr_cell_type_specific_class")) %>% 
    mutate(locus = case_when(is.na(locus) == TRUE ~ "None",
                            TRUE ~ locus),
           gene_family = case_when(is.na(gene_family) == TRUE ~ "None",
                            TRUE ~ locus),
           type = case_when(is.na(type) == TRUE ~ "None",
                            TRUE ~ type),
          )

Warning message in left_join(., combined_acrs_marker_info_subset, by = c("acr", :
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 17007 of `x` matches multiple rows in `y`.
ℹ Row 5 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship =
  "many-to-many"` to silence this warning.”


In [ ]:
regions_of_interest <- zm_acrs_TFs %>% 
    filter(type != "None")


all_other_regions <- zm_acrs_TFs %>% 
    filter(type == "None")



In [ ]:
GetGC <- function(bsgenome, gr){

  seqs <- BSgenome::getSeq(bsgenome, gr)
  return(as.numeric(Biostrings::letterFrequency(x = seqs, letters = "GC", as.prob = TRUE)))


}


In [169]:

ora_res <- list()
system.time({
    for (motif_id in unique(zm_acrs_TFs$TF_ID)) {
        subsample <- zm_acrs_TFs %>% 
            dplyr::filter(TF_ID == motif_id) # Filter for each motif_id

        xt <- xtabs(~ type + acr_cell_type_specific_class, data = subsample)

        # Replace zeros with ones in the contingency table
        xt <- apply(xt, c(1, 2), function(x) ifelse(x == 0, 1, x))
        
        print(xt[c("mesophyll", "None"), c("mesophyll", "broadly_accessible")])
        ft.ms <- fisher.test(xt[c("mesophyll", "None"), c("mesophyll", "broadly_accessible")], alternative = "greater")
        ft.bs <- fisher.test(xt[c("bundle_sheath", "None"), c("bundle_sheath", "broadly_accessible")], alternative = "greater")
        ft.bs_vs_ms <- fisher.test(xt[c("bundle_sheath", "mesophyll"), c("bundle_sheath", "mesophyll")], alternative = "greater")
        ft.ms_vs_bs <- fisher.test(xt[c("mesophyll", "bundle_sheath"), c("mesophyll", "bundle_sheath")], alternative = "greater")

        ora_res[[motif_id]] <- data.frame(
            "Motif.ID" = motif_id,
            "OR.ms" = unname(ft.ms$estimate), "pv.ms" = ft.ms$p.value,
            "OR.bs" = unname(ft.bs$estimate), "pv.bs" = ft.bs$p.value,
            "OR.bs_vs_ms" = unname(ft.bs_vs_ms$estimate), "pv.bs_vs_ms" = ft.bs_vs_ms$p.value,
            "OR.ms_vs_bs" = unname(ft.ms_vs_bs$estimate), "pv.ms_vs_bs" = ft.ms_vs_bs$p.value
        )
    }
})


ora_res <- as.data.frame(do.call("rbind",ora_res))
rownames(ora_res) <- NULL

# adjust p-values for multiple testing
ora_res$fdr.bs <- p.adjust(ora_res$pv.bs, method="BH")
ora_res$fdr.ms   <- p.adjust(ora_res$pv.ms, method="BH") 
ora_res$fdr.bs_vs_ms   <- p.adjust(ora_res$pv.bs_vs_ms, method="BH") 
ora_res$fdr.ms_vs_bs   <- p.adjust(ora_res$pv.ms_vs_bs, method="BH") 



           acr_cell_type_specific_class
type        mesophyll broadly_accessible
  mesophyll        17                 27
  None           1670              29923
           acr_cell_type_specific_class
type        mesophyll broadly_accessible
  mesophyll        24                 39
  None           2019              57950
           acr_cell_type_specific_class
type        mesophyll broadly_accessible
  mesophyll        51                124
  None           6475             145427
           acr_cell_type_specific_class
type        mesophyll broadly_accessible
  mesophyll        27                 43
  None           3185              57034
           acr_cell_type_specific_class
type        mesophyll broadly_accessible
  mesophyll         8                 38
  None           1826              50541
           acr_cell_type_specific_class
type        mesophyll broadly_accessible
  mesophyll        28                 87
  None           2522              71300
           acr_cell_ty

   user  system elapsed 
  0.854   0.041   0.898 

## Mona Lisa Method

In [86]:
zm_acrs

chr,start,end,acr_id,score
<chr>,<dbl>,<dbl>,<chr>,<chr>
chr1,34471,34972,scACR_1;broadly_accessible,NA
chr1,45954,46455,scACR_2;broadly_accessible,NA
chr1,123085,123586,scACR_3;broadly_accessible,NA
chr1,162544,163045,scACR_4;broadly_accessible,NA
chr1,165164,165665,scACR_5;broadly_accessible,NA
chr1,189283,189784,scACR_6;broadly_accessible,NA
chr1,199880,200381,scACR_7;broadly_accessible,NA
chr1,206262,206763,scACR_8;broadly_accessible,NA
chr1,252910,253411,scACR_9;broadly_accessible,NA


In [87]:
### Using MonaLisa for testing Motif Enrichment
zm_acrs_annotated <- as_tibble(as.data.frame(zm_acrs)) %>% 
    tidyr::separate(acr_id, into = c("acr", "acr_cell_type_specific_class"), sep = ";") %>%
    dplyr::mutate(acr_cell_type_specific_class = case_when(acr_cell_type_specific_class == "bundle_sheath,procambial_meristem" ~ "bundle_sheath",
                                                          acr_cell_type_specific_class == "bundle_sheath,procambium" ~ "bundle_sheath", 
                                                          TRUE ~ acr_cell_type_specific_class)) %>% 
    left_join(., combined_acrs_marker_info_subset, by = c("acr", "acr_cell_type_specific_class")) %>% 
    mutate(locus = case_when(is.na(locus) == TRUE ~ "None",
                            TRUE ~ locus),
           gene_family = case_when(is.na(gene_family) == TRUE ~ "None",
                            TRUE ~ locus),
           type = case_when(is.na(type) == TRUE ~ "None",
                            TRUE ~ type),
          ) %>% 
    dplyr::filter(acr_cell_type_specific_class %in% c("bundle_sheath", "broadly_accessible", "mesophyll", "bundle_sheath,mesophyll"))

zm_peaks_gr_annotated <- GRanges(zm_acrs_annotated, seqinfo=seqinfo(BSgenome.Zm_B73))
names(zm_peaks_gr_annotated) <- as.character(zm_peaks_gr_annotated)
zm_peaks_gr_annotated <- unique(zm_peaks_gr_annotated)


# get peak sequences, we'll need this later
zm_peak_seqs_annotated <- getSeq(BSgenome.Zm_B73, zm_peaks_gr_annotated) # <2s


# exclude peaks with any N's, to avoid warnings further down
hasNs    <- (alphabetFrequency(zm_peak_seqs_annotated)[,"N"]>0)
addmargins(table(hasNs)) # => only drops 5 sequences

zm_peak_seqs_annotated <- zm_peak_seqs_annotated[!hasNs]
zm_peak_seqs_annotated  <- zm_peak_seqs_annotated[!hasNs]



hasNs
FALSE   Sum 
47983 47983 

In [65]:
library(monaLisa)
# we first have to set up a way to define the "common" state as the background
zm_peaks_gr_annotated$acr_cell_type_specific_class <- setZeroBin(factor(zm_peaks_gr_annotated$acr_cell_type_specific_class), "broadly_accessible")

## Testing if the ACR class if signifigant in any way

# now run monaLisa
system.time( check_res_acr_type <- calcBinnedMotifEnrR(
  zm_peak_seqs_annotated,
  bins = zm_peaks_gr_annotated$acr_cell_type_specific_class,
  pwmL = pw_matrix_list_converted,
  background = "otherBins", 
  min.score = 3,
  BPPARAM =  BiocParallel::MulticoreParam(16)
) ) # ~40s on my laptop

   user  system elapsed 
 35.883   7.787  19.118 

In [74]:
# reformat the results for convenience
ora_res2_consider_acr_type <- cbind(
  rowData(check_res_acr_type)[,c("motif.id","motif.name")],
  2^assay(check_res_acr_type, "log2enr")[,c("bundle_sheath","mesophyll")],  # convert to OR
  10^(-1 * assay(check_res_acr_type, "negLog10Padj")[,c("bundle_sheath","mesophyll")]), # convert to p-val
  10^(-1 * assay(check_res_acr_type, "negLog10P")[,c("bundle_sheath","mesophyll")]) # convert to p-val
) 
colnames(ora_res2_consider_acr_type)[3:8] <- c("OR.bs", "OR.ms", 
                             "fdr.bs", "fdr.ms", 
                             "pval.bs", "pval.ms")


### Yuuuuup - very nice, aligns with expectations
as_tibble(ora_res2_consider_acr_type) %>% 
    arrange(OR.bs, OR.ms) %>% 
    dplyr::filter(pval.bs < .05 | pval.ms < .05)

motif.id,motif.name,OR.bs,OR.ms,fdr.bs,fdr.ms,pval.bs,pval.ms
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
BES1,BES1,0.9910325,1.0291767,1.000000e+00,1.899897e-02,7.404542e-01,3.166495e-03
C2C2_Dof,C2C2_Dof,1.0026643,0.9998445,1.706511e-01,1.000000e+00,4.875746e-02,6.318872e-01
bs_DOF,bs_DOF,1.0143999,0.9965168,6.151562e-06,1.000000e+00,5.948632e-07,8.644863e-01
ms_AGCTA,ms_AGCTA,1.0264024,1.0246645,9.960413e-02,3.865624e-02,2.371527e-02,7.823286e-03
ms_GACA,ms_GACA,1.0284745,1.0412858,9.401424e-02,1.808378e-04,2.126513e-02,2.152831e-05
BBRBPC,BBRBPC,1.0370805,1.0065790,5.626082e-04,6.427478e-01,7.367488e-05,2.325162e-01
NAC,NAC,1.0567678,1.0125019,2.365128e-02,6.427478e-01,4.505005e-03,2.414374e-01
bs_CATG,bs_CATG,1.1723881,1.1110174,6.265750e-08,6.151562e-06,2.983690e-09,4.128956e-07


In [66]:
library(monaLisa)
# we first have to set up a way to define the "common" state as the background
zm_peaks_gr_annotated$type <- setZeroBin(factor(zm_peaks_gr_annotated$type), "None")

# now run monaLisa
system.time( check_res_type <- calcBinnedMotifEnrR(
  zm_peak_seqs_annotated,
  bins = zm_peaks_gr_annotated$type,
  pwmL = pw_matrix_list_converted,
  background = "otherBins", 
  min.score = 3,
  BPPARAM =  BiocParallel::MulticoreParam(16)
) ) # ~40s on my laptop

   user  system elapsed 
 48.783   9.051  19.589 

In [91]:
# reformat the results for convenience
ora_res2_consider_type <- cbind(
  rowData(check_res_type)[,c("motif.id","motif.name")],
  2^assay(check_res_type, "log2enr")[,c("bundle_sheath","mesophyll", "mesophyll,bundle_sheath")],  # convert to OR
  10^(-1 * assay(check_res_type, "negLog10Padj")[,c("bundle_sheath","mesophyll","mesophyll,bundle_sheath")]), # convert to p-val
  10^(-1 * assay(check_res_type, "negLog10P")[,c("bundle_sheath","mesophyll", "mesophyll,bundle_sheath")]) # convert to p-val
) 
colnames(ora_res2_consider_type)[3:11] <- c("OR.bs", "OR.ms", "OR.ms_bs", 
                             "fdr.bs", "fdr.ms", "fdr.ms_bs", 
                             "pval.bs", "pval.ms", "pval.ms_bs")

as_tibble(ora_res2_consider_type) %>% 
    arrange(OR.bs, OR.ms) 

motif.id,motif.name,OR.bs,OR.ms,OR.ms_bs,fdr.bs,fdr.ms,fdr.ms_bs,pval.bs,pval.ms,pval.ms_bs
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AS2LOB,AS2LOB,0.9351690,0.9279546,1.0236580,1,1,1,0.9740776,0.98202857,0.49448293
HD-Zip_IV,HD-Zip_IV,0.9535660,0.8861025,1.0850942,1,1,1,0.8681568,0.99622827,0.18088532
HSF,HSF,0.9556872,1.0059945,1.0160428,1,1,1,0.7527697,0.53226094,0.53329654
CPP,CPP,0.9591760,0.9348583,1.1619745,1,1,1,0.8064409,0.92111661,0.04159156
ms_AGCTA,ms_AGCTA,0.9595508,1.0212335,1.0574985,1,1,1,0.8206621,0.42591781,0.29257470
bs_GCCGCC,bs_GCCGCC,0.9644805,1.0463189,1.0032460,1,1,1,0.7258991,0.33855958,0.61128779
bHLH_TCP,bHLH_TCP,0.9741285,1.0215303,0.9747614,1,1,1,0.9640359,0.25557447,0.97448692
SBP,SBP,0.9765223,0.9732956,0.9603793,1,1,1,0.8061510,0.82718454,0.88576430
MADS_MIKC,MADS_MIKC,0.9890780,0.9934723,0.9848422,1,1,1,0.9227107,0.88316849,0.89497279


In [ ]:
## Trying to Run MonaLisa on just Bs and Ms regions surrounding C4 genes,
## Asking if we see bias in some way shape or form. Clearly the signal gets swamped out otherwise

In [55]:
unique(zm_acrs_annotated$type)

[1] "None"                    "bundle_sheath"          
[3] "mesophyll"               "mesophyll,bundle_sheath"

In [76]:
### Using MonaLisa for testing Motif Enrichment
zm_acrs_in_c4 <- as_tibble(as.data.frame(zm_acrs)) %>% 
    tidyr::separate(acr_id, into = c("acr", "acr_cell_type_specific_class"), sep = ";") %>%
    dplyr::mutate(acr_cell_type_specific_class = case_when(acr_cell_type_specific_class == "bundle_sheath,procambial_meristem" ~ "bundle_sheath",
                                                          acr_cell_type_specific_class == "bundle_sheath,procambium" ~ "bundle_sheath", 
                                                          TRUE ~ acr_cell_type_specific_class)) %>% 
    left_join(., combined_acrs_marker_info_subset, by = c("acr", "acr_cell_type_specific_class")) %>% 
    mutate(locus = case_when(is.na(locus) == TRUE ~ "None",
                            TRUE ~ locus),
           gene_family = case_when(is.na(gene_family) == TRUE ~ "None",
                            TRUE ~ locus),
           type = case_when(is.na(type) == TRUE ~ "None",
                            TRUE ~ type),
          ) %>% 
    dplyr::filter(acr_cell_type_specific_class %in% c("bundle_sheath", "broadly_accessible", "mesophyll")) %>% 
    dplyr::filter(type != "None")

zm_acrs_in_c4_gr <- GRanges(zm_acrs_in_c4, seqinfo=seqinfo(BSgenome.Zm_B73))
names(zm_acrs_in_c4_gr) <- as.character(zm_acrs_in_c4_gr)
zm_acrs_in_c4_gr <- unique(zm_peaks_gr_annotated)


# get peak sequences, we'll need this later
zm_acrs_in_c4_sequences <- getSeq(BSgenome.Zm_B73, zm_acrs_in_c4_gr) # <2s


# exclude peaks with any N's, to avoid warnings further down
hasNs    <- (alphabetFrequency(zm_acrs_in_c4_sequences)[,"N"]>0)
addmargins(table(hasNs)) # => only drops 5 sequences

zm_acrs_in_c4_sequences <- zm_acrs_in_c4_sequences[!hasNs]
zm_acrs_in_c4_sequences  <- zm_acrs_in_c4_sequences[!hasNs]


hasNs
FALSE   Sum 
47756 47756 

In [84]:
library(monaLisa)

# now run monaLisa
system.time( check_motif_c4_type <- calcBinnedMotifEnrR(
  zm_acrs_in_c4_sequences,
  bins = zm_acrs_in_c4_gr$type,
  pwmL = pw_matrix_list_converted,
  min.score = 3,
  BPPARAM =  BiocParallel::MulticoreParam(16)
) ) # ~40s on my laptop

   user  system elapsed 
 36.599   8.800  20.165 

In [110]:
# reformat the results for convenience
c4_type_only_ora <- cbind(
  rowData(check_motif_c4_type)[,c("motif.id","motif.name")],
  2^assay(check_motif_c4_type, "log2enr")[,c("bundle_sheath","mesophyll")],  # convert to OR
  10^(-1 * assay(check_motif_c4_type, "negLog10Padj")[,c("bundle_sheath","mesophyll")]), # convert to p-val
  10^(-1 * assay(check_motif_c4_type, "negLog10P")[,c("bundle_sheath","mesophyll")]) # convert to p-val
) 
colnames(c4_type_only_ora)[3:8] <- c("OR.bs", "OR.ms", 
                             "fdr.bs", "fdr.ms", 
                             "pval.bs", "pval.ms")

c4_type_only_ora

DataFrame with 28 rows and 8 columns
                 motif.id   motif.name     OR.bs     OR.ms    fdr.bs    fdr.ms
              <character>  <character> <numeric> <numeric> <numeric> <numeric>
ARF                   ARF          ARF  1.000106  1.000297         1         1
AS2LOB             AS2LOB       AS2LOB  0.936416  0.928427         1         1
BBRBPC             BBRBPC       BBRBPC  1.036072  0.974336         1         1
BES1                 BES1         BES1  1.007741  1.038767         1         1
bHLH_TCP         bHLH_TCP     bHLH_TCP  0.974697  1.021001         1         1
...                   ...          ...       ...       ...       ...       ...
bs_GCCGCC       bs_GCCGCC    bs_GCCGCC  0.967986  1.039354         1         1
ms_GACGA         ms_GACGA     ms_GACGA  1.002611  1.004775         1         1
ms_AGCTA         ms_AGCTA     ms_AGCTA  0.962549  1.023548         1         1
ms_GACA           ms_GACA      ms_GACA  1.045623  1.094194         1         1
ms_CGTCGTCGT ms

### Running on Cell type Specific ACRs around C4 Genes Only, and some Braodly ACC ACRs

In [194]:
### Using MonaLisa for testing Motif Enrichment
zm_acrs_in_c4 <- zm_acrs_annotated %>% 
    dplyr::filter(type != "None" & acr_cell_type_specific_class != "broadly_accessible") %>% 
    dplyr::filter(acr_cell_type_specific_class %in% c("bundle_sheath", "broadly_accessible", "mesophyll"))  %>% 
    dplyr::filter(type == acr_cell_type_specific_class )

# zm_acrs_in_c4 %>% 
#     dplyr::group_by(type, acr_cell_type_specific_class )%>% 
#     summarise(counts = n())

all_null_acrs <- zm_acrs_annotated %>% 
    dplyr::filter(type == "None" & acr_cell_type_specific_class == "broadly_accessible") %>% 
    sample_n(500)

combined_vals <- bind_rows(zm_acrs_in_c4,all_null_acrs) %>% 
  mutate_at(vars(type, acr_cell_type_specific_class), factor)

x <- GRanges(combined_vals, seqinfo=seqinfo(BSgenome.Zm_B73))
names(x) <- as.character(x)
x <- unique(x)

# Convert 'type' to a factor
mcols(x)$acr_cell_type_specific_class <- factor(mcols(x)$acr_cell_type_specific_class)
mcols(x)$type <- factor(mcols(x)$type)


# get peak sequences, we'll need this later
zm_acrs_in_c4_sequences <- getSeq(BSgenome.Zm_B73, x) # <2s


# exclude peaks with any N's, to avoid warnings further down
hasNs    <- (alphabetFrequency(zm_acrs_in_c4_sequences)[,"N"]>0)
addmargins(table(hasNs)) # => only drops 5 sequences

zm_acrs_in_c4_sequences <- zm_acrs_in_c4_sequences[!hasNs]
zm_acrs_in_c4_sequences  <- zm_acrs_in_c4_sequences[!hasNs]


hasNs
FALSE   Sum 
  537   537 

In [195]:
library(monaLisa)
x$acr_cell_type_specific_class <- setZeroBin(factor(x$acr_cell_type_specific_class), "broadly_accessible")

# now run monaLisa
system.time( check_motif_c4_type <- calcBinnedMotifEnrR(
  zm_acrs_in_c4_sequences,
  bins = x$acr_cell_type_specific_class,
  pwmL = pw_matrix_list_converted,
  min.score = 3,
  background = "otherBins", 

  BPPARAM =  BiocParallel::MulticoreParam(16)
) ) # ~40s on my laptop

   user  system elapsed 
  3.089   2.315   1.552 

In [196]:
# reformat the results for convenience
c4_type_only_ora <- cbind(
  rowData(check_motif_c4_type)[,c("motif.id","motif.name")],
  2^assay(check_motif_c4_type, "log2enr")[,c("bundle_sheath","mesophyll")],  # convert to OR
  10^(-1 * assay(check_motif_c4_type, "negLog10Padj")[,c("bundle_sheath","mesophyll")]), # convert to p-val
  10^(-1 * assay(check_motif_c4_type, "negLog10P")[,c("bundle_sheath","mesophyll")]) # convert to p-val
) 
colnames(c4_type_only_ora)[3:8] <- c("OR.bs", "OR.ms", 
                             "fdr.bs", "fdr.ms", 
                             "pval.bs", "pval.ms")

In [197]:

c4_type_only_ora[c4_type_only_ora$pval.bs < .1, ]
c4_type_only_ora[c4_type_only_ora$pval.ms < .1, ]

DataFrame with 1 row and 8 columns
           motif.id  motif.name     OR.bs     OR.ms    fdr.bs    fdr.ms
        <character> <character> <numeric> <numeric> <numeric> <numeric>
ms_GACA     ms_GACA     ms_GACA   1.14611   1.08443         1         1
          pval.bs   pval.ms
        <numeric> <numeric>
ms_GACA 0.0586323  0.232239

DataFrame with 1 row and 8 columns
       motif.id  motif.name     OR.bs     OR.ms    fdr.bs    fdr.ms   pval.bs
    <character> <character> <numeric> <numeric> <numeric> <numeric> <numeric>
HSF         HSF         HSF    0.8349   1.18133         1         1  0.970034
      pval.ms
    <numeric>
HSF 0.0688643